In [2]:
pip install pyforest

  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14606 sha256=df5a135f74bc8ccba9086a0826dc4f49d954e8fc6b85064155ac57e30a53241b
  Stored in directory: /home/kurubal/.cache/pip/wheels/21/2d/19/8b5488f0caea6d892333413745640532d40d70e6b5351d7633
Successfully built pyforest
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyforest
import scipy.stats as stats
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, ElasticNetCV, Lasso, LassoCV
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle


In [2]:
df = pickle.load(open('payrol_cleaned.pkl', 'rb'))

In [3]:
df

,Office,Title,City,Biweeklt/hourly rate,Payrol type,Legislative entity
0,SENATOR TOBY ANN STAVISKY,COMMUNITY LIAISON,FLUSHING,1076.93,SA,SENATE EMPLOYEE
1,LEGISLATIVE MESSENGER SERVICE,MESSENGER,ALBANY,1173.93,RA,JOINT LEGISLATIVE EMPLOYEE
2,SENATE FINANCE COMMITTEE/MAJORITY,SENIOR ANALYST,ALBANY,2692.31,RA,SENATE EMPLOYEE
3,SENATOR JESSICA RAMOS,SR. DIR. LEG. AFFRS / COMMITTEE DIR.,ALBANY,2307.70,RA,SENATE EMPLOYEE
4,SENATE SERGEANT-AT-ARMS,SESSION SERGEANT-AT-ARMS,ALBANY,820.95,SA,SENATE EMPLOYEE
...,...,...,...,...,...,...
958,SENATOR TODD KAMINSKY,COMMUNITY LIAISON,ROCKVILLE CENTRE,1538.47,RA,SENATE EMPLOYEE
959,SS/PRODUCTION SERVICES,DATA ANALYST,ALBANY,2180.24,RA,SENATE EMPLOYEE
960,SS/PRODUCTION SERVICES,MAILING OPERATOR II,ALBANY,1770.23,RA,SENATE EMPLOYEE
961,SENATE TECHNOLOGY SERVICES,SENIOR OFFICE SUPPORT REPRESENTATIVE,ALBANY,2917.25,RA,SENATE EMPLOYEE


In [4]:

df.shape

(963, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963 entries, 0 to 962
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Office                963 non-null    object 
 1   Title                 963 non-null    object 
 2   City                  963 non-null    object 
 3   Biweeklt/hourly rate  963 non-null    float64
 4   Payrol type           963 non-null    object 
 5   Legislative entity    963 non-null    object 
dtypes: float64(1), object(5)
memory usage: 45.3+ KB


In [6]:
df.isnull().any()

Office                  False
Title                   False
City                    False
Biweeklt/hourly rate    False
Payrol type             False
Legislative entity      False
dtype: bool

In [7]:
df.value_counts("Title")

Title
COMMUNITY LIAISON                     43
CHIEF OF STAFF                        43
LEGISLATIVE DIRECTOR                  28
ADMINISTRATIVE ASSISTANT              26
COMMUNICATIONS DIRECTOR               23
                                      ..
DRUPAL ARCHITECT                       1
DRIVER                                 1
DOWNSTATE OPERATIONS COORDINATOR       1
DOCUMENT ROOM MANAGER                  1
EXEC DIR OF SEN DEM INTERN PROGRAM     1
Length: 406, dtype: int64

In [10]:
df.Title.value_counts()

CHIEF OF STAFF                          43
COMMUNITY LIAISON                       43
LEGISLATIVE DIRECTOR                    28
ADMINISTRATIVE ASSISTANT                26
COMMUNICATIONS DIRECTOR                 23
                                        ..
ALBANY OFFICE DIRECTOR                   1
EXECUTIVE ASSISTANT/COMMITTEE CLERK/     1
DIRECTOR & COUNSEL OF EXTERNAL RELAT     1
DEPUTY DIR OF SENATE POLICY GROUP        1
MAIL SELECTION SPECIALIST                1
Name: Title, Length: 406, dtype: int64

In [13]:
len(df.Title.value_counts() < 2)

406

In [8]:
df.Title.unique()

array(['COMMUNITY LIAISON', 'MESSENGER', 'SENIOR ANALYST',
       'SR. DIR. LEG. AFFRS / COMMITTEE DIR.', 'SESSION SERGEANT-AT-ARMS',
       'CHIEF OF STAFF', 'ADMINISTRATIVE ASSISTANT', 'DISTRICT DIRECTOR',
       'COMMUNICATIONS DIRECTOR', 'TECHNICAL SUPPORT REPRESENTATIVE',
       'EXTERNAL RELATIONS ASSOCIATE', 'LEGISLATIVE DIRECTOR',
       'DISTRICT POLICY & ORGANIZING DIRECTO', 'POSTAL CLERK',
       'MESSENGER APPRENTICE', 'ASSOCIATE COUNSEL', 'EXECUTIVE AIDE',
       'DISTRICT REPRESENTATIVE', 'EXTERNAL RELATIONS LEGISLATIVE AIDE',
       'SESSION AIDE', 'OFFICE MANAGER', 'CONSTITUENT RELATIONS AIDE',
       'DIRECTOR OF OPERATIONS', 'DEPUTY CHIEF OF STAFF', 'COUNSEL',
       'SENIOR ADVISOR', 'EXECUTIVE ASSISTANT',
       'DISTRICT OFFICE COORDINATOR',
       'CONSTITUENT SERVICES REPRESENTATIVE',
       'DIRECTOR OF COMMUNITY RELATIONS', 'TRANSLATOR',
       'EMPLOYEE BENEFITS ASSISTANT', 'ATTORNEY',
       'EXECUTIVE ASSISTANT/ALBANY OFFICE MA', 'SENATE FELLOW',
       'LEG

In [9]:
list_of_least_title = []
for i in df.Title.unique():
  if df.Title.value_counts()[i] < 2 :
    list_of_least_title.append(i)

In [10]:
least_title_index = []
for i in df.Title:
  if i in list_of_least_title:
    least_title_index.append(df[df.Title == i].index[0])
    #print(df[df.Title == i].index)

df.iloc[least_title_index]

,Office,Title,City,Biweeklt/hourly rate,Payrol type,Legislative entity
3,SENATOR JESSICA RAMOS,SR. DIR. LEG. AFFRS / COMMITTEE DIR.,ALBANY,2307.70,RA,SENATE EMPLOYEE
10,SENATE TECHNOLOGY SERVICES,TECHNICAL SUPPORT REPRESENTATIVE,ALBANY,1307.70,RA,SENATE EMPLOYEE
13,SENATOR JESSICA RAMOS,DISTRICT POLICY & ORGANIZING DIRECTO,EAST ELMHURST,2000.00,RA,SENATE EMPLOYEE
20,MAJORITY CONFERENCE SERVICES,EXTERNAL RELATIONS LEGISLATIVE AIDE,ALBANY,2115.39,SA,SENATE EMPLOYEE
31,SENATOR SUSAN SERINO,DISTRICT OFFICE COORDINATOR,PUTNAM VALLEY,2307.70,RA,SENATE EMPLOYEE
...,...,...,...,...,...,...
947,SENATOR JAMES SANDERS,TEAM COUNSEL,ALBANY,2500.00,RA,SENATE EMPLOYEE
948,SENATOR ANNA M. KAPLAN,CHIEF OF STAFF/COUNSEL,ALBANY,4615.39,RA,SENATE EMPLOYEE
950,SENATE M&O/POST OFFICE,POSTMASTER,ALBANY,2628.45,RA,SENATE EMPLOYEE
952,MAJORITY COUNSEL/PROGRAM,POLICY ASSISTANT,ALBANY,1538.47,RA,SENATE EMPLOYEE


## Iteretion 2


In [11]:
df1 = df.copy()
df1

,Office,Title,City,Biweeklt/hourly rate,Payrol type,Legislative entity
0,SENATOR TOBY ANN STAVISKY,COMMUNITY LIAISON,FLUSHING,1076.93,SA,SENATE EMPLOYEE
1,LEGISLATIVE MESSENGER SERVICE,MESSENGER,ALBANY,1173.93,RA,JOINT LEGISLATIVE EMPLOYEE
2,SENATE FINANCE COMMITTEE/MAJORITY,SENIOR ANALYST,ALBANY,2692.31,RA,SENATE EMPLOYEE
3,SENATOR JESSICA RAMOS,SR. DIR. LEG. AFFRS / COMMITTEE DIR.,ALBANY,2307.70,RA,SENATE EMPLOYEE
4,SENATE SERGEANT-AT-ARMS,SESSION SERGEANT-AT-ARMS,ALBANY,820.95,SA,SENATE EMPLOYEE
...,...,...,...,...,...,...
958,SENATOR TODD KAMINSKY,COMMUNITY LIAISON,ROCKVILLE CENTRE,1538.47,RA,SENATE EMPLOYEE
959,SS/PRODUCTION SERVICES,DATA ANALYST,ALBANY,2180.24,RA,SENATE EMPLOYEE
960,SS/PRODUCTION SERVICES,MAILING OPERATOR II,ALBANY,1770.23,RA,SENATE EMPLOYEE
961,SENATE TECHNOLOGY SERVICES,SENIOR OFFICE SUPPORT REPRESENTATIVE,ALBANY,2917.25,RA,SENATE EMPLOYEE


In [12]:
df1.columns = df1.columns.str.lower()

In [13]:
df1

,office,title,city,biweeklt/hourly rate,payrol type,legislative entity
0,SENATOR TOBY ANN STAVISKY,COMMUNITY LIAISON,FLUSHING,1076.93,SA,SENATE EMPLOYEE
1,LEGISLATIVE MESSENGER SERVICE,MESSENGER,ALBANY,1173.93,RA,JOINT LEGISLATIVE EMPLOYEE
2,SENATE FINANCE COMMITTEE/MAJORITY,SENIOR ANALYST,ALBANY,2692.31,RA,SENATE EMPLOYEE
3,SENATOR JESSICA RAMOS,SR. DIR. LEG. AFFRS / COMMITTEE DIR.,ALBANY,2307.70,RA,SENATE EMPLOYEE
4,SENATE SERGEANT-AT-ARMS,SESSION SERGEANT-AT-ARMS,ALBANY,820.95,SA,SENATE EMPLOYEE
...,...,...,...,...,...,...
958,SENATOR TODD KAMINSKY,COMMUNITY LIAISON,ROCKVILLE CENTRE,1538.47,RA,SENATE EMPLOYEE
959,SS/PRODUCTION SERVICES,DATA ANALYST,ALBANY,2180.24,RA,SENATE EMPLOYEE
960,SS/PRODUCTION SERVICES,MAILING OPERATOR II,ALBANY,1770.23,RA,SENATE EMPLOYEE
961,SENATE TECHNOLOGY SERVICES,SENIOR OFFICE SUPPORT REPRESENTATIVE,ALBANY,2917.25,RA,SENATE EMPLOYEE


In [14]:
df1["title"] = df1.title.str.lower()

In [15]:
df1.title.str.contains("dir|manager").sum()

242

In [16]:
df1.title.str.contains("senior|chief|supervisor").sum()

147

In [17]:
df1.title.str.contains("coordinator|liaison|assistant").sum()

260

In [18]:
df1.title.str.contains("analyst|engineer|operator").sum()

50

In [19]:
df1["director_manager"] = df1.title.str.contains("dir|manager")
df1["senior_chief_supervisor"] = df1.title.str.contains("senior|chief|supervisor")
df1["coordinator_liason_assistant"] = df1.title.str.contains("coordinator|liaison|assistant")
df1["analyst_engineer_operator"] = df1.title.str.contains("analyst|engineer|operator")

In [20]:
df1

,office,title,city,biweeklt/hourly rate,payrol type,legislative entity,director_manager,senior_chief_supervisor,coordinator_liason_assistant,analyst_engineer_operator
0,SENATOR TOBY ANN STAVISKY,community liaison,FLUSHING,1076.93,SA,SENATE EMPLOYEE,False,False,True,False
1,LEGISLATIVE MESSENGER SERVICE,messenger,ALBANY,1173.93,RA,JOINT LEGISLATIVE EMPLOYEE,False,False,False,False
2,SENATE FINANCE COMMITTEE/MAJORITY,senior analyst,ALBANY,2692.31,RA,SENATE EMPLOYEE,False,True,False,True
3,SENATOR JESSICA RAMOS,sr. dir. leg. affrs / committee dir.,ALBANY,2307.70,RA,SENATE EMPLOYEE,True,False,False,False
4,SENATE SERGEANT-AT-ARMS,session sergeant-at-arms,ALBANY,820.95,SA,SENATE EMPLOYEE,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
958,SENATOR TODD KAMINSKY,community liaison,ROCKVILLE CENTRE,1538.47,RA,SENATE EMPLOYEE,False,False,True,False
959,SS/PRODUCTION SERVICES,data analyst,ALBANY,2180.24,RA,SENATE EMPLOYEE,False,False,False,True
960,SS/PRODUCTION SERVICES,mailing operator ii,ALBANY,1770.23,RA,SENATE EMPLOYEE,False,False,False,True
961,SENATE TECHNOLOGY SERVICES,senior office support representative,ALBANY,2917.25,RA,SENATE EMPLOYEE,False,True,False,False


In [21]:
df2 = df1.drop("title", axis=1)

In [22]:
df2

,office,city,biweeklt/hourly rate,payrol type,legislative entity,director_manager,senior_chief_supervisor,coordinator_liason_assistant,analyst_engineer_operator
0,SENATOR TOBY ANN STAVISKY,FLUSHING,1076.93,SA,SENATE EMPLOYEE,False,False,True,False
1,LEGISLATIVE MESSENGER SERVICE,ALBANY,1173.93,RA,JOINT LEGISLATIVE EMPLOYEE,False,False,False,False
2,SENATE FINANCE COMMITTEE/MAJORITY,ALBANY,2692.31,RA,SENATE EMPLOYEE,False,True,False,True
3,SENATOR JESSICA RAMOS,ALBANY,2307.70,RA,SENATE EMPLOYEE,True,False,False,False
4,SENATE SERGEANT-AT-ARMS,ALBANY,820.95,SA,SENATE EMPLOYEE,False,False,False,False
...,...,...,...,...,...,...,...,...,...
958,SENATOR TODD KAMINSKY,ROCKVILLE CENTRE,1538.47,RA,SENATE EMPLOYEE,False,False,True,False
959,SS/PRODUCTION SERVICES,ALBANY,2180.24,RA,SENATE EMPLOYEE,False,False,False,True
960,SS/PRODUCTION SERVICES,ALBANY,1770.23,RA,SENATE EMPLOYEE,False,False,False,True
961,SENATE TECHNOLOGY SERVICES,ALBANY,2917.25,RA,SENATE EMPLOYEE,False,True,False,False


In [23]:
df2.office.unique()

array(['SENATOR TOBY ANN STAVISKY', 'LEGISLATIVE MESSENGER SERVICE',
       'SENATE FINANCE COMMITTEE/MAJORITY', 'SENATOR JESSICA RAMOS',
       'SENATE SERGEANT-AT-ARMS', 'SENATOR JAMAAL BAILEY',
       'SENATOR JOSEPH A. GRIFFO', 'SENATOR JOHN BROOKS',
       'SENATE TECHNOLOGY SERVICES', 'MAJORITY CONFERENCE SERVICES',
       "SENATOR THOMAS F. O'MARA", 'SENATE M&O/POST OFFICE',
       'MAJORITY COUNSEL/PROGRAM', 'SENATOR JAMES SANDERS',
       'SENATOR DAVID S. CARLUCCI', 'SENATOR KEVIN M. THOMAS',
       'SS/SENATE OFFICE SERVICES', 'SENATOR JOSEPH E. ROBACH',
       'SENATOR FREDERICK J. AKSHAR II', 'SENATOR ALESSANDRA BIAGGI',
       'SENATOR JAMES SKOUFIS', 'MAJORITY COMMUNICATIONS',
       'SENATOR SUSAN SERINO', 'SENATOR JULIA C. SALAZAR',
       'SENATE PERSONNEL OFFICE', 'SENATOR LUIS SEPULVEDA',
       'SENATOR TODD KAMINSKY', 'SEN STUDENT PROGRAM OFF/FELLOWS',
       'SENATOR JOSE M. SERRANO', 'SENATOR SHELLEY MAYER',
       'SENATOR DIANE J. SAVINO', 'SENATOR MICHAEL H. 

In [24]:
df2.office.str.contains("SENATE").sum()

138

In [25]:
df2.office.str.contains("SENATOR").sum()

561

In [26]:
df2.office.str.contains("LEGISLATIVE").sum()

38

In [27]:
df2["senate"] = df2.office.str.contains("SENATE")
df2["senator"] = df2.office.str.contains("SENATOR")
df2["legislative"] = df2.office.str.contains("LEGISLATIVE")


In [28]:
df2

,office,city,biweeklt/hourly rate,payrol type,legislative entity,director_manager,senior_chief_supervisor,coordinator_liason_assistant,analyst_engineer_operator,senate,senator,legislative
0,SENATOR TOBY ANN STAVISKY,FLUSHING,1076.93,SA,SENATE EMPLOYEE,False,False,True,False,False,True,False
1,LEGISLATIVE MESSENGER SERVICE,ALBANY,1173.93,RA,JOINT LEGISLATIVE EMPLOYEE,False,False,False,False,False,False,True
2,SENATE FINANCE COMMITTEE/MAJORITY,ALBANY,2692.31,RA,SENATE EMPLOYEE,False,True,False,True,True,False,False
3,SENATOR JESSICA RAMOS,ALBANY,2307.70,RA,SENATE EMPLOYEE,True,False,False,False,False,True,False
4,SENATE SERGEANT-AT-ARMS,ALBANY,820.95,SA,SENATE EMPLOYEE,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
958,SENATOR TODD KAMINSKY,ROCKVILLE CENTRE,1538.47,RA,SENATE EMPLOYEE,False,False,True,False,False,True,False
959,SS/PRODUCTION SERVICES,ALBANY,2180.24,RA,SENATE EMPLOYEE,False,False,False,True,False,False,False
960,SS/PRODUCTION SERVICES,ALBANY,1770.23,RA,SENATE EMPLOYEE,False,False,False,True,False,False,False
961,SENATE TECHNOLOGY SERVICES,ALBANY,2917.25,RA,SENATE EMPLOYEE,False,True,False,False,True,False,False


In [29]:
df2["legislative entity"].value_counts()

SENATE EMPLOYEE               926
JOINT LEGISLATIVE EMPLOYEE     37
Name: legislative entity, dtype: int64

In [30]:
df2.drop("legislative entity", axis=1,inplace=True)
df2

,office,city,biweeklt/hourly rate,payrol type,director_manager,senior_chief_supervisor,coordinator_liason_assistant,analyst_engineer_operator,senate,senator,legislative
0,SENATOR TOBY ANN STAVISKY,FLUSHING,1076.93,SA,False,False,True,False,False,True,False
1,LEGISLATIVE MESSENGER SERVICE,ALBANY,1173.93,RA,False,False,False,False,False,False,True
2,SENATE FINANCE COMMITTEE/MAJORITY,ALBANY,2692.31,RA,False,True,False,True,True,False,False
3,SENATOR JESSICA RAMOS,ALBANY,2307.70,RA,True,False,False,False,False,True,False
4,SENATE SERGEANT-AT-ARMS,ALBANY,820.95,SA,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
958,SENATOR TODD KAMINSKY,ROCKVILLE CENTRE,1538.47,RA,False,False,True,False,False,True,False
959,SS/PRODUCTION SERVICES,ALBANY,2180.24,RA,False,False,False,True,False,False,False
960,SS/PRODUCTION SERVICES,ALBANY,1770.23,RA,False,False,False,True,False,False,False
961,SENATE TECHNOLOGY SERVICES,ALBANY,2917.25,RA,False,True,False,False,True,False,False


In [31]:
df2.drop("office", axis=1,inplace=True)
df2

,city,biweeklt/hourly rate,payrol type,director_manager,senior_chief_supervisor,coordinator_liason_assistant,analyst_engineer_operator,senate,senator,legislative
0,FLUSHING,1076.93,SA,False,False,True,False,False,True,False
1,ALBANY,1173.93,RA,False,False,False,False,False,False,True
2,ALBANY,2692.31,RA,False,True,False,True,True,False,False
3,ALBANY,2307.70,RA,True,False,False,False,False,True,False
4,ALBANY,820.95,SA,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
958,ROCKVILLE CENTRE,1538.47,RA,False,False,True,False,False,True,False
959,ALBANY,2180.24,RA,False,False,False,True,False,False,False
960,ALBANY,1770.23,RA,False,False,False,True,False,False,False
961,ALBANY,2917.25,RA,False,True,False,False,True,False,False


In [32]:
df2 = pd.get_dummies(df2, drop_first=True)

<IPython.core.display.Javascript object>

In [33]:
df2

,biweeklt/hourly rate,director_manager,senior_chief_supervisor,coordinator_liason_assistant,analyst_engineer_operator,senate,senator,legislative,city_ASTORIA,city_BAY SHORE,...,city_SMITHTOWN,city_STATEN ISLAND,city_SYOSSET,city_SYRACUSE,city_UTICA,city_WATERTOWN,city_WILLIAMSVILLE,city_YONKERS,payrol type_SA,payrol type_TE
0,1076.93,False,False,True,False,False,True,False,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1173.93,False,False,False,False,False,False,True,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2692.31,False,True,False,True,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2307.70,True,False,False,False,False,True,False,0,0,...,0,0,0,0,0,0,0,0,0,0
4,820.95,False,False,False,False,True,False,False,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,1538.47,False,False,True,False,False,True,False,0,0,...,0,0,0,0,0,0,0,0,0,0
959,2180.24,False,False,False,True,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
960,1770.23,False,False,False,True,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
961,2917.25,False,True,False,False,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
X = df2.drop("biweeklt/hourly rate",axis=1)
y = df2["biweeklt/hourly rate"]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

<IPython.core.display.Javascript object>

# Ridge Regression

In [37]:
alpha_space = np.linspace(0.01, 20, 300)

<IPython.core.display.Javascript object>

In [38]:
ridgecv = RidgeCV(alphas=alpha_space, cv=10).fit(X_train, y_train)

In [39]:
ridgecv.alpha_

5.492207357859532

In [40]:
index= ["Actual", "Prediction"]

In [41]:
ypred = ridgecv.predict(X_test)

In [42]:
ytrain_pred = ridgecv.predict(X_train)

In [43]:
pd.DataFrame([ypred,y_test], index= ["Actual", "Prediction"]).T.sample(10)

<IPython.core.display.Javascript object>

,Actual,Prediction
79,1157.000745,392.31
84,2119.789487,1538.47
94,2217.915406,2884.62
69,2558.806141,2500.00
67,2119.789487,2712.31
23,1699.696269,961.54
59,1521.683161,1571.43
57,1676.016148,2153.85
24,1599.490040,1423.08
18,870.946791,1057.70


In [44]:
ridgecv.best_score_

0.45492686728673987

In [45]:
ridgecv.score(X_train, y_train)

0.4919533175802203

In [46]:
ridgecv.score(X_test, y_test)

0.3617952116573703

* Ridge Regression Failed(Underfitting)

# Lasso Regression

In [47]:
lasso_cv_model = LassoCV(alphas = alpha_space,  cv = 10, max_iter = 1000, n_jobs =-1).fit(X_train, y_train)

In [48]:
lasso_cv_model.alpha_

2.349966555183946

In [49]:
ypred = lasso_cv_model.predict(X_test)

In [50]:
ytrain_pred = lasso_cv_model.predict(X_train)

In [51]:
pd.DataFrame([ypred,y_test], index= ["Actual", "Prediction"]).T.sample(10)

<IPython.core.display.Javascript object>

,Actual,Prediction
28,1699.642782,1884.62
45,2867.982244,4615.39
71,2799.748188,2307.70
96,2125.703519,1625.54
92,1500.714910,1224.00
29,2521.048833,2115.39
91,1947.721723,1634.62
74,1947.721723,2115.39
77,1691.900719,1730.77
34,1829.450259,1538.47


In [52]:
r2_score(y_train, ytrain_pred)

0.47873101067710366

In [53]:
r2_score(y_test, ypred)

0.34174824000238235

* Lasso Regression Failed (Underfitting)

# KNN Regressor

In [54]:
from sklearn.neighbors import KNeighborsRegressor

In [55]:
model = KNeighborsRegressor()

In [56]:
model.fit(X_train,y_train)

KNeighborsRegressor()

In [57]:
ypred = model.predict(X_test)

In [58]:
ytrain_pred = model.predict(X_train)

In [59]:
pd.DataFrame([ypred,y_test], index= ["Actual", "Prediction"]).T.sample(10)

<IPython.core.display.Javascript object>

,Actual,Prediction
62,1990.872,1774.97
61,1592.312,1538.47
46,3653.084,2384.62
37,1269.236,2788.47
72,2266.954,2041.67
74,1990.872,2115.39
32,1677.694,888.00
31,1976.444,2884.62
57,1583.698,2153.85
28,2386.592,1884.62


In [60]:
r2_score(y_train, ytrain_pred)

0.5544405763545509

In [61]:
r2_score(y_test, ypred)

0.15401586904167786

In [62]:
# KNN GridSearchCV

In [63]:
from sklearn.model_selection import GridSearchCV

In [64]:
knn_grid = KNeighborsRegressor()

In [65]:
k_values= range(1,30)

In [66]:
param_grid = {"n_neighbors":k_values ,"p":[1,2], "weights": ['uniform',"distance"]}

In [67]:
knn_grid_model = GridSearchCV(knn_grid, param_grid, cv=10, scoring= 'r2')

In [68]:
knn_grid_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 30), 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='r2')

In [69]:
knn_grid_model.best_estimator_

KNeighborsRegressor(n_neighbors=16, p=1)

In [70]:
knn_grid_model.best_params_

{'n_neighbors': 16, 'p': 1, 'weights': 'uniform'}

In [71]:
ypred = knn_grid_model.predict(X_test)

In [72]:
ytrain_pred = knn_grid_model.predict(X_train)

In [73]:
pd.DataFrame([ypred,y_test], index= ["Actual", "Prediction"]).T.sample(10)

<IPython.core.display.Javascript object>

,Actual,Prediction
21,2262.589375,2769.24
51,2296.328750,2167.08
25,1132.110000,769.24
29,2447.119375,2115.39
62,2000.624375,1774.97
59,1517.544375,1571.43
80,2020.776875,1538.47
52,1908.123750,692.31
30,1982.370000,2000.00
72,2296.328750,2041.67


In [74]:
r2_score(y_train, ytrain_pred)

0.5363943531326361

In [75]:
r2_score(y_test, ypred)

0.27701303368740815

* KNN Failed(underfittin)

## What would you suggest about this project? Writedown below.